In [30]:
import sys
import random
import copy
import numpy as np

class State:
    def __init__(self, route:[], distance:int=0):
        self.route = route
        self.distance = distance

    def deepcopy(self):
        return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))
    
    def update_distance(self, matrix, home):
        self.distance = 0
        from_index = home
        for i in range(len(self.route)):
            self.distance += matrix[from_index][self.route[i]]
            from_index = self.route[i]
        self.distance += matrix[from_index][home]

class City:
    def __init__(self, index:int, distance:int):
        self.index = index
        self.distance = distance
    def __lt__(self, other):
         return self.distance < other.distance
        
def eprob(p):
    return p > random.uniform(0.0, 1.0)
def exp_schedule(k=20, lam=0.005, limit=1000):
    return lambda t: (k * np.exp(-lam * t) if t < limit else 0)

def initial_solution(matrix:[], home:int):
    route = []
    from_index = home
    length = len(matrix) - 1
    while len(route) < length:
        row = matrix[from_index]
        cities = {}
        for i in range(len(row)):
            cities[i] = City(i, row[i])
        del cities[home]
        for i in route:
            del cities[i]
        sorted = list(cities.values())
        sorted.sort()
        from_index = sorted[0].index
        route.append(from_index)
    state = State(route)
    state.update_distance(matrix, home)
    return state

def neighbor_states(matrix:[], home:int, state:State, search:float=0.01):
    neighbor_state = state.deepcopy()
    for i in range(len(neighbor_state.route)):
        if(random.random() < search):
            j = int(random.random() * len(state.route))
            tourist1 = neighbor_state.route[i]
            tourist2 = neighbor_state.route[j]
            neighbor_state.route[i] = tourist2
            neighbor_state.route[j] = tourist1
    neighbor_state.update_distance(matrix, home)
    return neighbor_state

def tsp(matrix:[], home:int, initial_state:State, search:float=0.01, schedule=exp_schedule()):
    best_state = initial_state
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return best_state
        neighbor = neighbor_states(matrix, home, best_state, search)
        excess = best_state.distance - neighbor.distance
        if excess > 0 or eprob(np.exp(excess / T)):
            best_state = neighbor
def main():
    # Cities to travel
    tourist = ['Jaipur', 'Udaipur', 'Jaisalmer', 'Jodhpur', 'Bikaner', 'Ranthambore', 'Pushkar', 'Mount Abu', 'Chittorgarh', 'Ajmer', 'Bharatpur', 'Alwar', 'Khumbhalgarh', 'Shekhawati', 'Bundi', 'Neemrana', 'Ranakpur', 'Nathdwara', 'Garadiya Mahadev', 'Fatehpur']
    tourist_indexes = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    home = 2
    distances = [[0, 245, 71, 101, 163, 137, 240, 21, 257, 87, 142, 214, 197, 189, 200, 50, 79, 69, 420, 300],
            [24, 0, 174, 152, 83, 124, 95, 259, 40, 158, 137, 35, 57, 143, 262, 84, 67, 79, 100, 320],
            [71, 174, 0, 35, 92, 80, 173, 85, 185, 26, 94, 145, 126, 100, 120, 320, 32, 40, 50, 69],
            [101, 152, 35, 0, 70, 86, 139, 112, 158, 46, 105, 128, 98, 421, 369, 70, 100, 20, 120, 40],
            [163, 83, 92, 70, 0, 66, 102, 176, 94, 79, 87, 58, 37, 100, 50, 60, 70, 200, 180, 320],
            [137, 124, 80, 86, 66, 0, 168, 155, 176, 54, 22, 88, 99, 101, 202, 322, 75, 183, 500, 120],
            [240, 95, 173, 139, 102, 168, 0, 249, 67, 172, 189, 111, 70, 51, 64, 79, 84, 97, 99, 241],
            [21, 259, 85, 112, 176, 155, 249, 0, 269, 103, 160, 230, 209,56, 64, 246, 36, 45, 26, 180],
            [257, 40, 185, 158, 94, 176, 67, 269, 0, 174, 164, 65, 60, 56, 64, 246, 36, 45, 26, 180],
            [87, 158, 26, 46, 79, 54, 172, 103, 174, 0, 67, 127, 116, 327, 254, 188, 290, 190, 350, 210],
            [142, 137, 94, 105, 87, 22, 189, 160, 164, 67, 0, 101, 120, 167, 142, 195, 184, 147, 200, 260],
            [214, 35, 145, 128, 58, 88, 111, 230, 65, 127, 101, 0, 50, 200, 180, 156, 543, 123, 327, 480],
            [197, 57, 126, 98, 37, 99, 70, 209, 60, 116, 120, 50, 0, 450, 500, 124, 290, 320, 410, 56],
            [189, 143, 100, 421, 100, 101, 51, 56, 70, 327, 167, 200, 450, 0, 260, 240, 350, 120, 100, 32],
            [200, 262, 120, 369, 50, 202, 64, 156, 80, 254, 142, 180, 500, 260, 0, 150, 244, 84, 230, 300],
            [50, 84, 320, 70, 60, 322, 79, 246, 84, 188, 195, 156, 124, 240, 150, 0, 250, 423, 510, 200],
            [79, 67, 32, 100, 70, 75, 84, 36, 89, 290, 184, 543, 290, 350, 244, 250, 0, 321, 214, 87],
            [69, 79, 40, 20, 200, 183, 97, 45, 73, 190, 147, 123, 320, 120, 84, 423, 321, 0, 561, 251],
            [420, 100, 50, 120, 180, 500, 99, 26, 49, 350, 200, 327, 410, 100, 230, 510, 214, 561, 0, 130],
            [300, 320, 69, 40, 320, 120, 241, 180, 180, 210, 260, 480, 56, 32, 300, 200, 87, 251, 130, 0]]
    # Run simulated annealing to find a better solution
    state = initial_solution(distances, home)
    state = tsp(distances, home, state, 0.1)
    print('Path obtained: ')
    print(tourist[home], end='')
    for i in range(0, len(state.route)):
       print('  ' + tourist[state.route[i]], end='')
    print('  ' + tourist[home], end='')
    print(f'\nTotal distance: {state.distance} kms')
    print()
if __name__ == "__main__": 
    main()

Path obtained: 
Jaisalmer  Ajmer  Jodhpur  Nathdwara  Mount Abu  Jaipur  Neemrana  Bikaner  Khumbhalgarh  Alwar  Udaipur  Chittorgarh  Garadiya Mahadev  Fatehpur  Shekhawati  Pushkar  Bundi  Bharatpur  Ranthambore  Ranakpur  Jaisalmer
Total distance: 1004 kms

